In [ ]:
## 使用Qwen-coder，对保险客户数据表进行SQL查询

In [9]:
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role
import time
import pandas as pd
import re

# 设置API密钥
dashscope.api_key = "sk-e37b6feeb62f492bb01828119d588fc4" 

# 封装模型响应函数
def get_response(messages):
    response = dashscope.Generation.call(
        model='qwen-coder-plus',
        messages=messages,
        result_format='message'  # 将输出设置为message形式
    )
    return response

# 从模型响应中提取SQL代码
def get_sql_code(response):
    # 查找```sql和```之间的内容
    pattern = r'```sql(.*?)```'
    match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # 如果没有找到```sql标记，尝试查找任何```之间的内容
        pattern = r'```(.*?)```'
        match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            # 如果没有找到任何代码块，返回整个响应
            return response.output.choices[0].message.content

# 得到sql
def get_sql(query):
    start_time = time.time()
    sys_prompt = """我正在编写SQL，以下是数据库中的数据表和字段，请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL，如果有多个查询语句，请尝试合并为一个。编写SQL请采用```sql
    """
    user_prompt = f"""-- language: SQL
### Question: {query}
### Input: {create_sql}
### Response:
Here is the SQL query I have generated to answer the question `{query}`:
```sql
"""

    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = get_response(messages)
    return response


######## 需要人工设置 ########
save_file = f'sql_result_qwen_coder.xlsx'
qa_file = './data/qa_list-2.txt'                  # QA测试题
sql_file = './data/create_sql.txt'           # SQL数据表

In [10]:
# 读取 SQL数据表
with open(sql_file, 'r', encoding='utf-8') as file:
    create_sql = file.read()
# 读取 SQL问题列表
with open(qa_file, 'r', encoding='utf-8') as file:
    qa_list = file.read()
qa_list = qa_list.split('=====')

# 保存SQL结果
sql_list = []
markdown_list = []
time_list = []
for qa in qa_list:
    query = qa
    query = query.replace('\n', '')
    print(query)
    start_time = time.time()
    # 请求生成sql
    #content, prompt_len = get_sql(query)
    response = get_sql(query)
    use_time = round(time.time()-start_time, 2)
    time_list.append(use_time)
    print('SQL生成时间：', use_time)
    print(response)
    print('response=', response.output.choices[0].message.content)
    # 提取生成的SQL
    sql = get_sql_code(response)
    print('SQL: {}'.format(sql))
    sql_list.append(sql)

获取所有客户的姓名和联系电话。
SQL生成时间： 2.16
{"status_code": 200, "request_id": "a6d1dab4-dc64-9a89-ba26-7a7e69eb14d1", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "```sql\nSELECT Name, PhoneNumber\nFROM CustomerInfo;\n```\n\nThis SQL statement selects the `Name` and `PhoneNumber` fields from the `CustomerInfo` table, which contains all the customer information including their names and contact numbers."}}]}, "usage": {"input_tokens": 817, "output_tokens": 48, "total_tokens": 865}}
response= ```sql
SELECT Name, PhoneNumber
FROM CustomerInfo;
```

This SQL statement selects the `Name` and `PhoneNumber` fields from the `CustomerInfo` table, which contains all the customer information including their names and contact numbers.
SQL: SELECT Name, PhoneNumber
FROM CustomerInfo;
找出所有已婚客户的客户ID和配偶姓名。
SQL生成时间： 9.53
{"status_code": 200, "request_id": "4ef91d30-8b76-9b12-bd75-2bcabdf83e34", "code": ""

In [11]:
result = pd.DataFrame(columns=['QA', 'SQL', 'time'])
result['QA'] = qa_list
result['SQL'] = sql_list
result['time'] = time_list
result.to_excel(save_file, index=False)
result

TypeError: Cannot convert numpy.ndarray to numpy.ndarray